# CSE547 - Colab 2
## Frequent Pattern Mining in Spark

Adapted from Stanford CS246

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [0]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 217.8MB 60kB/s 
     |████████████████████████████████| 204kB 42.3MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=95caf0945c8941800e476af38a7700d0186fd3b377b042c70c577fb27c9153c3
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark
openjdk-8-jdk-headless is already the newest version (8u242-b08-0ubuntu3~18.04).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
id='1dhi1F78ssqR8gE6U-AgB80ZW7V_9snX4'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('products.csv')

id='1KZBNEaIyMTcsRV817us6uLZgm-Mii8oU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('order_products__train.csv')

If you executed the cells above, you should be able to see the dataset we will need for this Colab under the "Files" tab on the left panel.

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F

Let's initialize the Spark context.

In [0]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [0]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-04-12 15:56:57--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.192.78.186, 3.95.144.123, 52.5.232.178, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.192.78.186|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  34.5MB/s    in 0.4s    

2020-04-12 15:56:58 (34.5 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://1e252051.ngrok.io


### Your task

If you run successfully the setup stage, you are ready to work with the **3 Million Instacart Orders** dataset. In case you want to read more about it, check the [official Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) about it, a concise [schema description](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b) of the dataset, and the [download page](https://www.instacart.com/datasets/grocery-shopping-2017).

In this Colab, we will be working only with a small training dataset (~131K orders) to perform fast Frequent Pattern Mining with the FP-Growth algorithm.

In [0]:
products = spark.read.csv('products.csv', header=True, inferSchema=True)
orders = spark.read.csv('order_products__train.csv', header=True, inferSchema=True)

In [0]:
products.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- department_id: string (nullable = true)



In [0]:
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)



Column<b'product_id'>

Use the Spark Dataframe API to join 'products' and 'orders', so that you will be able to see the product names in each transaction (and not only their ids).  Then, group by the orders by 'order_id' to obtain one row per basket (i.e., set of products purchased together by one customer). 

In [0]:
joined = orders.join(products, orders.product_id==products.product_id)\
  .drop('product_id')\
  .drop('add_to_cart_order')\
  .drop('reordered')\
  .drop('aisle_id')\
  .drop('department_id')\
  .groupby('order_id')\
  .agg(collect_list('product_name').alias('products'))
joined.take(1)

[Row(order_id=1342, products=['Bag of Organic Bananas', 'Seedless Cucumbers', 'Organic Mandarins', 'Organic Strawberries', 'Versatile Stain Remover', 'Pink Lady Apples', 'Chicken Apple Sausage', 'Raw Shrimp'])]

In this Colab we will explore [MLlib](https://spark.apache.org/mllib/), Apache Spark's scalable machine learning library. Specifically, you can use its implementation of the [FP-Growth](https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html#fp-growth) algorithm to perform efficiently Frequent Pattern Mining in Spark.
Use the Python example in the documentation, and train a model with 

```minSupport=0.01``` and ```minConfidence=0.5```



In [0]:
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol='products', minSupport=0.01, minConfidence=0.5)
model = fpGrowth.fit(joined)


Compute how many Frequent Itemsets and Association Rules were generated by running FP-growth.


In [0]:
# Count frequent itemsets.
print(model.freqItemsets.count())

# Count generated association rules.
print(model.associationRules.count())

120
0


Now retrain the FP-growth model changing only 
```minsupport=0.001``` 
and compute how many Frequent Itemsets and Association Rules were generated.

In [0]:
fpGrowth = FPGrowth(itemsCol='products', minSupport=0.001, minConfidence=0.5)
model2 = fpGrowth.fit(joined)

# Count frequent itemsets.
print(model2.freqItemsets.count())

# Count generated association rules.
print(model2.associationRules.count())

4444
11


Now, for 
```minsupport=0.001```, 
visualize the top Frequent Itemsets and Association Rules.

In addition for
```minsupport=0.001```, 
calculate the "interest" of the Association Rules. 

(Hint: One way to do this by joining the Frequent Itemsets with the Association Rules)




In [0]:
# Count frequent itemsets.
model2.freqItemsets.sort(desc('freq')).show()

# Count generated association rules.
model2.associationRules.show()

+--------------------+-----+
|               items| freq|
+--------------------+-----+
|            [Banana]|18726|
|[Bag of Organic B...|15480|
|[Organic Strawber...|10894|
|[Organic Baby Spi...| 9784|
|       [Large Lemon]| 8135|
|   [Organic Avocado]| 7409|
|[Organic Hass Avo...| 7293|
|      [Strawberries]| 6494|
|             [Limes]| 6033|
|[Organic Raspberr...| 5546|
|[Organic Blueberr...| 4966|
|[Organic Whole Milk]| 4908|
|  [Organic Cucumber]| 4613|
|  [Organic Zucchini]| 4589|
|[Organic Yellow O...| 4290|
|    [Organic Garlic]| 4158|
|[Seedless Red Gra...| 4059|
|         [Asparagus]| 3868|
|[Organic Grape To...| 3823|
| [Organic Red Onion]| 3818|
+--------------------+-----+
only showing top 20 rows

+--------------------+--------------------+------------------+------------------+
|          antecedent|          consequent|        confidence|              lift|
+--------------------+--------------------+------------------+------------------+
|[Organic Kiwi, Or...|[Bag of Or

In [0]:
num_baskets = joined.count()

joined_model = model2.freqItemsets.\
  join(model2.associationRules,
    model2.freqItemsets.items==model2.associationRules.consequent 
).withColumn('interest',abs(col('confidence') - col('freq')/num_baskets ))\
.sort(desc('interest'))
  
joined_model.toPandas().show()

                       items   freq  ...      lift  interest
0   [Bag of Organic Bananas]  15480  ...  5.072272  0.480445
1   [Bag of Organic Bananas]  15480  ...  4.635331  0.428895
2   [Bag of Organic Bananas]  15480  ...  4.627719  0.427997
3   [Bag of Organic Bananas]  15480  ...  4.587387  0.423239
4   [Bag of Organic Bananas]  15480  ...  4.504745  0.413489
5   [Bag of Organic Bananas]  15480  ...  4.477905  0.410322
6   [Bag of Organic Bananas]  15480  ...  4.416854  0.403119
7   [Bag of Organic Bananas]  15480  ...  4.382495  0.399066
8   [Bag of Organic Bananas]  15480  ...  4.357585  0.396127
9                   [Banana]  18726  ...  3.753633  0.392995
10  [Bag of Organic Bananas]  15480  ...  4.278898  0.386843

[11 rows x 7 columns]


In [0]:
a= joined_model.toPandas()
print(a.head(5))
print([a for a in a.head(1)['antecedent']])

                      items   freq  ...      lift  interest
0  [Bag of Organic Bananas]  15480  ...  5.072272  0.480445
1  [Bag of Organic Bananas]  15480  ...  4.635331  0.428895
2  [Bag of Organic Bananas]  15480  ...  4.627719  0.427997
3  [Bag of Organic Bananas]  15480  ...  4.587387  0.423239
4  [Bag of Organic Bananas]  15480  ...  4.504745  0.413489

[5 rows x 7 columns]
[['Organic Raspberries', 'Organic Hass Avocado', 'Organic Strawberries']]


To conclude, go to Gradescope and read the remaining questions. We will ask you to inspect the resulting dataframes, and report a few results.

Once you obtained the desired results, **submit your solution for this Colab**!